In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import time
import random
from ebooklib import epub #http://docs.sourcefabric.org/projects/ebooklib/en/latest/tutorial.html#creating-epub
from IPython.display import display, clear_output

In [ ]:
class Page:
    def __init__(self, url, base_url):
        self.url = url
        self.base_url = base_url
        req = requests.get(self.url, headers)
        self.soup = BeautifulSoup(req.content)
        
    def get_next_url(self):
        next_url = self.soup.find('a', string=re.compile(css_nexttext))['href']
        next_url = base_url + next_url
        return next_url

    def get_booktitle(self):
        """soup.find('span', class_='shuming')"""
        title = self.soup.find(**css_booktitle).text
        title = self._clean_text(title)
        return title

    def get_author(self):
        """soup.find('h1', id='timu')"""
        author = self.soup.find(**css_author).text
        author = self._clean_text(author)
        return author
    
    def get_chaptitle(self):
        """soup.find('h1', id='timu')"""
        title = self.soup.find(**css_chaptitle).text
        title = self._clean_text(title)
        return title
    
    def get_content(self):
        """soup.find('h1', id='timu')"""
        # find all 'p'
        # content = self.soup.find_all(**css_content)
        # content = [x.get_text() for x in content]
        # content = '\n'.join(content)
        content = self.soup.find('div', class_=re.compile('cont')).get_text('\n')
#         # print(page.soup.find('div', class_=re.compile('cont')).get_text("\n"))
        # clean
        content = content.strip()
        content = content.replace('\u3000', '\n')
        content = content.replace('\xa0', '\n')
        content = content.replace('<br/>', '\n')
        
        content = re.sub(r'\n+', '\n', content)
        content = '\t'.join(('\n'+content.lstrip()).splitlines(keepends=True))
        
        # add chaptitle
        chaptitle = self.get_chaptitle()
        t = time.localtime()
        current_time = time.strftime("%H:%M", t)
        content_update_info = f"{current_time}  {chaptitle} {len(content)}"
        try:
            content_display.update(content_update_info)
        except:
            print(content_update_info)
        content = chaptitle + '\n' + content
        
        # html
        content = content.replace('\n', '<br>') #.sub(r'\n', '<br>', content) #
        content = content.replace('\t', '&emsp;') #.sub(r'\t', '&emsp;', content) #
        return content
        
    def _clean_text(self, text):
        text = text.replace('小說', '')
        text = text.replace('作者', '')
        text = text.replace('：', ' ')
        text = text.replace(':', ' ')
        text = re.sub(r' +', ' ', text)
        text = text.strip()
        return text
    
"""
utils function
"""    
def run_test():
    page = Page(url, base_url)
    print('Booktitle:\t', page.get_booktitle())
    print('Author:\t\t', page.get_author())
    print('Chap:\t\t', page.get_chaptitle())
    print('Content length:\t', end=' ')
    page.get_content()
    print('Next url:\t', page.get_next_url())
    
def create_book(first_page, language='zh'):
    booktitle = first_page.get_booktitle()
    print('Booktitle:\t', booktitle)

    # define book
    book = epub.EpubBook()
    book.set_title(booktitle)
    book.set_language(language)
    try:
        author = first_page.get_author()
        print('Author:\t\t', author)
        book.add_author(author)
    except:
        print('No author added')
    return book

def create_chap(page):
    chaptitle = page.get_chaptitle()
    chap = epub.EpubHtml(title=chaptitle, file_name=chaptitle + '.xhtml')
    chap.set_content(page.get_content())
    # chap.content = content
    return chap

def write_book(book, booktitle, toc, spine):
    # toc
    toc = tuple(toc)
    book.toc = toc

    # add default NCX and Nav file
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # spine = tuple(spine)
    book.spine = ['nav', *spine]

    # write to the file
    epub.write_epub('book/' + booktitle + '.epub', book, {})

# Config

In [ ]:
page_limit = 0
# https://tw.uukanshu.com/b/16893/52390.html
url = """
https://tw.uukanshu.com/b/62531/1389.html
"""
url = url.strip()
base_url = 'https://tw.uukanshu.com'

# css
css_booktitle    = {'name': 'span', 'class_': 'shuming'}
css_author       = {'name': 'span', 'class_': 'author'}
css_chaptitle    = {'name': 'h1', 'id': 'timu'}
css_content      = {'name': 'p'}
css_nexttext     = '下一章'

# optional
language = 'zh'
file_format = "epub"

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

run_test()

# Save book

In [ ]:
first_page = Page(url, base_url)
booktitle = first_page.get_booktitle()
author = first_page.get_author()
cnt = 0

"""
declare file format
"""
if file_format == 'epub':
    book = create_book(first_page, language=language)
    # define toc
    toc = []
    spine = []
else:
    filename = booktitle + ".txt"
    f = open(filename, "a+")
    
# print current chapter
content_display = display('Getting chapter content...', display_id=True)

In [ ]:
# print('Booktitle:\t', booktitle)
# print('Author:\t\t', author)
"""
start scraping
"""
while cnt < page_limit or page_limit == 0:
    time.sleep(random.randint(1, 4))
    cnt += 1
    
    page = Page(url, base_url)
    
    # save as epub or txt file
    if file_format == 'epub':
        chap = create_chap(page)

        # add chap
        book.add_item(chap)
        spine.append(chap)
        # create toc
        link = epub.Link(chap.file_name, chap.title, chap.id)
        toc.append(link)
    else:
        f.write(page.get_content())

    # get next url
    if page_limit == 1: break
    try:
        next_url = page.get_next_url()
        url = next_url
    except:
        print("End of url")
        break

In [ ]:
"""
save file
"""
if file_format == 'epub':
    write_book(book, booktitle, toc, spine)
else:
    f.close()

print("====\nDONE")

In [ ]:
assert False, "breakpoint"

# Test

In [ ]:
page = Page(url, base_url)
print(page.get_content())
# page.get_content()

In [ ]:
print(page.soup.find('div', class_=re.compile('cont')).get_text("\n"))
# page.soup.find('div', class_=re.compile('cont')).get_text("\n")